**Hruday Vishal Kanna Anand**

**1006874517**

In [ ]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

In [ ]:
#starter code
env = gym.make("CartPole-v0")
env.reset()
prev_screen = env.render(mode='rgb_array')
plt.imshow(prev_screen)

for i in range(5000):
  action = env.action_space.sample()
  print("step i",i,"action=",action)
  obs, reward, done, info = env.step(action)
  print("obs=",obs,"reward=",reward,"done=",done,"info=",info)
  screen = env.render(mode='rgb_array')
  
  plt.imshow(screen)
  ipythondisplay.clear_output(wait=True)
  ipythondisplay.display(plt.gcf())
  
  if done:
    break
    
ipythondisplay.clear_output(wait=True)
env.close()
print("Iterations that were run:",i)

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in futu

DependencyNotInstalled: ignored

In [ ]:
#defining the action space and the number of states and actions
actionSpace = [0,1]
numStates = 11 ** 4
numActions = env.action_space.n

In [ ]:
# descretizing the observation space so it is easier for the model to handle the problem
def createBins():
  bins = np.zeros((4, 10))
  bins[0] = np.linspace(-1.5, 1.5, 10)
  bins[1] = np.linspace(-2.5, 2.5, 10)
  bins[2] = np.linspace(-.5, .5, 10)
  bins[3] = np.linspace(-3, 3, 10)

  return bins

In [ ]:
# function to assign a observation to a perticular descrete state
def assignBins(obs, bins):
  state = np.zeros(4)
  for i in range(4):
    state[i] = np.digitize(obs[i], bins[i])
  return state

In [ ]:
#initilizing the action value function
def initQ():
  Q = {}

  for i in range(11):
    for j in range(11):
      for k in range(11):
        for l in range(11):

          Q[(i,j,k,l)] = {}
          for action in range(env.action_space.n):
            Q[(i,j,k,l)][action] = float(0)
  return Q

In [ ]:
#initilize a random deterministic target policy
def initTargetPolicy():
  pi = {}

  for i in range(11):
    for j in range(11):
      for k in range(11):
        for l in range(11):
          pi[(i,j,k,l)]= np.random.choice(actionSpace)
  return pi

In [ ]:
#initilize a behaviour soft policy based on the target policy
def initBehaviourPolicy(epsilon,pi):
  b = {}

  for i in range(11):
    for j in range(11):
      for k in range(11):
        for l in range(11):
          #if np.random.choice(actionSpace, p=[0.9,0.1])==0:
            if pi[(i,j,k,l)]==0:  
              b[(i,j,k,l)]= [1-epsilon,epsilon]
            else:
              b[(i,j,k,l)]= [epsilon,1-epsilon]
            '''else:
            if pi[(i,j,k,l)]==0:
              b[(i,j,k,l)]= [epsilon,1-epsilon]
            else:'''

  return b

In [ ]:
#initilize C
def initC():
  C = {}

  for i in range(11):
    for j in range(11):
      for k in range(11):
        for l in range(11):
          C[(i,j,k,l)] = {}
          for action in range(env.action_space.n):
            C[(i,j,k,l)][action] = 0
  return C

In [ ]:
bins= createBins()

In [ ]:
q= initQ()

In [ ]:
pi=initTargetPolicy()

In [ ]:
c= initC()

In [ ]:
# A function to handle the running of a single episode it maintains a lists of states, actions and rewards that it returns 
def episode(env, policy,bins):
  obs = env.reset()
  rewards = []
  actions = []
  states = []
  while True:
    state = tuple(assignBins(obs, bins))
    action = np.random.choice(actionSpace, p=policy[state])
    actions.append(action)
    states.append(state)

    obs, reward, done, info = env.step(action)
    rewards.append(reward)
    if done == True:
        break

  return states, actions, rewards

In [ ]:
# the main program the iterates over episodes to calculate estimate the value of q 
# using the behaviour policy and improve the policy with off policy monte carlo
env = gym.make('CartPole-v0')

#constants chosen by us
eps = 0.1
gamma = 0.7

#iterate 20000 times
for i in range(20000):
  G = 0 #initilize reuturn for each episode
  W = 1 #initilize W for each episode
  states, actions, rewards = episode(env, b, bins) # run one episode
  print("episode:", i)

  # calucluate values of return(G), C, q, choose the greediest action for the target policy and update W
  # by iterating in reverse through all the states actions and rewards
  for i, (state, action, reward) in enumerate(zip(reversed(states), reversed(actions), reversed(rewards))):
    G = reward + G * gamma
    c[state][action] += W
    q[state][action] += (W / c[state][action]) * (G - q[state][action])
    values=[]
    for a in actionSpace:
      values.append(q[state][a])
    values=np.array(values)
    max= np.argmax(values)
    pi[state]= actionSpace[max]
    if action != pi[state]:
      break;
    
    W *= 1 / b[state][action]

In [ ]:
# testing the learned target policy on the enviroment again
obs = env.reset()
rewards = []
actions = []
states = []

while True:
    state = tuple(assignBins(obs, bins))
    action = pi[state]
    actions.append(action)
    states.append(state)

    obs, reward, done, info = env.step(action)
    rewards.append(reward)
    if done == True:
        break

print(len(states))

We can see that the cart pole is only stable for 200 iterations and this isnt a very desired result this may be due to either the wrong hyper parameters for learning.

it can also be due to a biased behaviour policy or insufficient episodes as the state action sapce is very large and he model has not seen all variations.

but it can be seen that our learned policy is much better than a random policy from before. the random policy only ran for 10 iterations where as the learned policy ran for 200 iterations.

In [ ]:
#saving the logs of the test
file1 = open("outputlogs.txt", "w") 
str1 = repr(states)
file1.write("states = " + str1 + "\n")
str2 = repr(actions)
file1.write("actions = " + str2 + "\n")
str3 = repr(rewards)
file1.write("rewards = " + str3 + "\n")
file1.close()